<a href="https://colab.research.google.com/github/junghyun9108/2025_DL/blob/main/13%EA%B0%95_RBM_%EC%8B%A4%EC%8A%B5_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [23]:
class SimpleRBM:
    def __init__(self, n_visible, n_hidden):
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.weights = np.random.randn(n_visible, n_hidden) * 0.1 # 가중치 W
        self.visible_bias = np.zeros(n_visible) # 가시층 바이어스
        self.hidden_bias = np.zeros(n_hidden) # 은닉층 바이어스

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def train(self, data, learning_rate = 0.1 , n_epochs = 100):
        for _ in range(n_epochs):
            for v in data:
                #은닉층 계산
                h_prob = self.sigmoid(np.dot(v, self.weights) + self.hidden_bias)
                h = np.random.binomial(1, h_prob)

                #재구성 계산
                v_recon = np.random.binomial(1, self.sigmoid(np.dot(h, self.weights.T) + self.visible_bias))
                h_recon_prob = self.sigmoid(np.dot(v_recon, self.weights) + self.hidden_bias)

                #가중치 업데이트
                self.weights += learning_rate * (np.outer(v,h) - np.outer(v_recon, h_recon_prob))
                self.visible_bias += learning_rate * (v - v_recon)
                self.hidden_bias += learning_rate * (h - h_recon_prob)

    def recommend(self , user_ratings , top_n = 3):
        h_prob = self.sigmoid(np.dot(user_ratings, self.weights) + self.hidden_bias)
        v_prob = self.sigmoid(np.dot(h_prob, self.weights.T) + self.visible_bias)
        recommendations = np.argsort(v_prob)[::-1][:top_n]
        return recommendations

In [24]:
#영화 데이터 (0: 안봄, 1: 봄)
movies = [
    "매트릭스", "스타워즈", "인스터스텔라", "블레이드 러너", "에이리언", #sci-fi
    "노팅힐", "러브 액츄얼리", "브리짓", "그녀", "500일의 썸머" # 로맨틱 코미디
]

users_data = np.array([
    [1,1,1,1,1,0,0,0,0,0],
    [1,1,1,0,1,0,0,0,1,0],
    [0,0,0,0,0,1,1,1,1,1],
    [0,0,0,1,0,1,1,1,1,0],
    [1,1,0,0,0,1,1,0,0,1],
    [1,0,1,1,0,0,1,0,1,0]
])

In [25]:
#RBM 모델 생성 및 학습
rbm = SimpleRBM(n_visible = 10 , n_hidden = 5)
rbm.train(users_data, n_epochs= 5000)

In [35]:
# 새로운 사용자의 영화 시청 기록
new_user_scifi = np.array([1,1,1,0,0,0,0,0,0,0])
new_user_romcom = np.array([0,0,0,0,0,1,1,1,0,0])

In [31]:
def print_recommendations(user_ratings, user_type):
    recommendations = rbm.recommend(user_ratings, top_n = 3)
    print(f"\n({user_type} 선호 사용자 추천 영화:)")
    for i in recommendations:
        print(f"- {movies[i]} ({'Sci-fi' if i < 5 else '로맨틱 코매디'})")


In [32]:
print_recommendations(new_user_scifi,"Sci-fi")


(Sci-fi 선호 사용자 추천 영화:)
- 매트릭스 (Sci-fi)
- 인스터스텔라 (Sci-fi)
- 스타워즈 (Sci-fi)


In [36]:
print_recommendations(new_user_romcom,"로맨틱 코미디")


(로맨틱 코미디 선호 사용자 추천 영화:)
- 러브 액츄얼리 (로맨틱 코매디)
- 노팅힐 (로맨틱 코매디)
- 브리짓 (로맨틱 코매디)
